In [1]:
import re

labortext = """
(18.08.2022 12:29:00)

Hämatologie: Leukozyten: 5.2 [4.37 - 9.68] n*1000/µl; Erythrozyten: 4.01 [3.92 - 5.08] n*10E6/µl; Hämoglobin: 11.6 [12.0 - 14.6] g/dl; Hämatokrit: 33.9 [36.6 - 44.0] %; MCV: 84.5 [77.70 - 93.70] fl; MCH (HbE): 28.9 [25.30 - 30.90] pg; MCHC: 34.2 [31.00 - 34.10] g/dl; Thrombozyten: 256 [186 - 353] n*1000/µl; Mittleres Plättchenvolumen: 9.5 [9.6 - 12.0] fl;

(18.08.2022 09:28:00)

Klinische Chemie: Natrium: 144 [135 - 145] mmol/l; Kalium: 4.3 [3.5 - 5.1] mmol/l; Calcium: 2.5 [2.0 - 2.7] mmol/l; anorg. Phosphat: 1.26 [0.87 - 1.45] mmol/l; glomerul. Filtrationsr. CKD-EP: 102 ml/min /1,73qm; glomerul. Filtrationsr. (MDRD): 92 ml/min /1,73qm; Creatinin: 0.80 [0 - 0.95] mg/dl; Harnstoff: 23.0 [10 - 50] mg/dl; Gesamt-Bilirubin: 0.2 [0.1 - 1.2] mg/dl; GOT (ASAT): 24.3 [10 - 35] U/l; GPT (ALAT): 20.5 [10 - 35] U/l; GGT: 12.0 [<= 40] U/l; Alk. Phosphatase: 90 [35 - 105] U/l; Lactat Dehydrogenase: 211 [<= 250] U/l; Lipase: 12 [13 - 60] U/l;

Gerinnung: Thromboplastinzeit n. Quick: 87 [80 - 126] %; Ratio int. norm.: 1.06 [0.85 - 1.18] ; PTT: 18.0 [21 - 31] s;

Hämatologie: Leukozyten: 5.7 [4.37 - 9.68] n*1000/µl; Erythrozyten: 4.74 [3.92 - 5.08] n*10E6/µl; Hämoglobin: 13.8 [12.0 - 14.6] g/dl; Hämatokrit: 40.1 [36.6 - 44.0] %; MCV: 84.6 [77.70 - 93.70] fl; MCH (HbE): 29.1 [25.30 - 30.90] pg; MCHC: 34.4 [31.00 - 34.10] g/dl; Thrombozyten: 301 [186 - 353] n*1000/µl; Mittleres Plättchenvolumen: 10.0 [9.6 - 12.0] fl;

Akutbestimmungen / TDM / Drogen: C-reaktives Protein: < 0.10 [0 - 0.5] mg/dl;
"""

labortext = labortext.replace("\n", " ")
labortext = labortext.strip()
REMOVE_LAB_FLAGS = [
    "Akutbestimmungen / TDM / Drogen:",
    "Hämatologie:",
    "Klinische Chemie:",
    "Gerinnung:",
]
for flag in REMOVE_LAB_FLAGS:
    labortext = labortext.replace(flag, "")

In [2]:
RE_TIMESTAMP = r"(\(\d{2}\.\d{2}.\d{4} \d{2}:\d{2}:\d{2}\))"

def get_timestamps(text, re_timestamp = RE_TIMESTAMP):
    return re.findall(re_timestamp, text)

def split_lab_text_by_date(text, re_timestamp = RE_TIMESTAMP):
    timestamps = get_timestamps(text, re_timestamp)
    return {timestamp: text.split(timestamp)[1] for timestamp in timestamps}

In [71]:
timestamps = get_timestamps(labortext)
from datetime import datetime as dt


_text = labortext
for timestamp in timestamps:
    _text = _text.replace(timestamp, "__XX__XX__XX__")

_text_split = _text.split("__XX__XX__XX__")
_text_split = [_.strip() for _ in _text_split if _ ]
labs = {dt.strptime(timestamps[i], "(%d.%m.%Y %H:%M:%S)"): _ for i, _ in enumerate(_text_split)}

In [79]:
lab_dicts = []

LOOKUP_UNITS = [
    "ml/min /1,73qm",
    "glomerul. Filtrationsr. (MDRD)",
    "glomerul. Filtrationsr. CKD-EP",
]

LOOKUP_REPLACES = {
    "C-reaktives Protein": [(">", ""), ("<", ""), (" ", "")],
}

def read_text_lab_value(value):
    l_type = value.split(":")[0]
    if l_type in LOOKUP_REPLACES:
        for replace in LOOKUP_REPLACES[l_type]:
            value = value.replace(replace[0], replace[1])

        
    if "[" in value and "]" in value:
        l_value = re.findall(r"(:\s*\d*\.*\d*\s*\[)", value)[0].replace(":", "").replace("[", "")
        ref = re.findall(r"(\[.+\])", value)[0].replace("[", "").replace("]", "").strip()

        if "<=" in ref:
            ref = ref.replace("<=", "0 -")
        if "<" in value:
            ref = ref.replace("<=", "0 -")
        ref = ref.split("-")
        ref_lower = float(ref[0])
        ref_upper = float(ref[1])
        unit = value.split("]")[-1].strip()

    else:
        l_value = re.findall(r"(:\s*\d*\.*\d*\s*)", value)[0].replace(": ", "").replace("[", "")
        ref_lower = None
        ref_upper = None
        unit = None
        for _ in LOOKUP_UNITS:
            if _ in value:
                unit = _

    return {
        "type": l_type,
        "value": l_value,
        "ref_lower": ref_lower,
        "ref_upper": ref_upper,
        "unit": unit
    }

for key, value in labs.items():
    lab_values = value.split(";")
    lab_values = [_.strip() for _ in lab_values if _]
    _new = []
    for _ in lab_values:
        _new.append(read_text_lab_value(_))
        _new[-1].update({"timestamp": key})

    lab_dicts.extend(_new)


C-reaktivesProtein:0.10[0-0.5]mg/dl


In [81]:
import pandas as pd


df = pd.DataFrame.from_dict(lab_dicts)
df
refs = {}
for _type in df["type"].unique():
    lower = df.loc[df["type"] == _type, "ref_lower"].min()
    upper = df.loc[df["type"] == _type, "ref_upper"].max()
    unit = df.loc[df["type"] == _type, "unit"].unique()[0]
    refs[_type] = {"lower": lower, "upper": upper, "unit": unit}

df = pd.pivot(df, index="type", columns=["timestamp"], values="value")
ref_df = pd.DataFrame.from_dict(refs, orient="index")


df = df.merge(ref_df, left_index=True, right_index=True)

df



,2022-08-18 09:28:00,2022-08-18 12:29:00,lower,upper,unit
Alk. Phosphatase,90,NaN,35.00,105.00,U/l
C-reaktives Protein,0.10,NaN,0.00,0.50,mg/dl
Calcium,2.5,NaN,2.00,2.70,mmol/l
Creatinin,0.80,NaN,0.00,0.95,mg/dl
Erythrozyten,4.74,4.01,3.92,5.08,n*10E6/µl
GGT,12.0,NaN,0.00,40.00,U/l
GOT (ASAT),24.3,NaN,10.00,35.00,U/l
GPT (ALAT),20.5,NaN,10.00,35.00,U/l
Gesamt-Bilirubin,0.2,NaN,0.10,1.20,mg/dl
Harnstoff,23.0,NaN,10.00,50.00,mg/dl


In [82]:
units = {
    index: row["unit"] for index, row in df.iterrows()
}

units

{'Alk. Phosphatase': 'U/l',
 'C-reaktives Protein': 'mg/dl',
 'Calcium': 'mmol/l',
 'Creatinin': 'mg/dl',
 'Erythrozyten': 'n*10E6/µl',
 'GGT': 'U/l',
 'GOT (ASAT)': 'U/l',
 'GPT (ALAT)': 'U/l',
 'Gesamt-Bilirubin': 'mg/dl',
 'Harnstoff': 'mg/dl',
 'Hämatokrit': '%',
 'Hämoglobin': 'g/dl',
 'Kalium': 'mmol/l',
 'Lactat Dehydrogenase': 'U/l',
 'Leukozyten': 'n*1000/µl',
 'Lipase': 'U/l',
 'MCH (HbE)': 'pg',
 'MCHC': 'g/dl',
 'MCV': 'fl',
 'Mittleres Plättchenvolumen': 'fl',
 'Natrium': 'mmol/l',
 'PTT': 's',
 'Ratio int. norm.': '',
 'Thromboplastinzeit n. Quick': '%',
 'Thrombozyten': 'n*1000/µl',
 'anorg. Phosphat': 'mmol/l',
 'glomerul. Filtrationsr. (MDRD)': 'glomerul. Filtrationsr. (MDRD)',
 'glomerul. Filtrationsr. CKD-EP': 'glomerul. Filtrationsr. CKD-EP'}